In [18]:
import os
import fsspec
from fsspec.implementations.local import LocalFileSystem
from s3fs.core import S3FileSystem
from urllib.parse import urlparse

def get_https_from_s3_url(bucket_name, path):
    return f"https://{bucket_name}.s3.amazonaws.com{path}"

def get_fs(url):
    pu = urlparse(url)
    if pu.scheme in ['http','https'] and pu.netloc.endswith('.s3.amazonaws.com'):
        # Convert S3 HTTP URLs (which do not support list operations) back to S3 REST API
        fs = fsspec.filesystem('s3')
        p = pu.netloc.split('.')[0] + pu.path
        u = url
    else:
        fs = fsspec.filesystem(pu.scheme)
        p = pu.netloc + pu.path
        if pu.scheme in ['s3']:
            components = pu.path.split(os.sep)
            bucket = pu.netloc
            path = os.path.join(*components[1:]) if len(components) > 1 else ''
            u = get_https_from_s3_url(bucket, pu.path)
        else:
            u = url
    return fs, p, u

url1 = "s3://janelia-flylight-imagery/Fly-eFISH/EASI-FISH_NP_SS"
fs1, fs1root, url = get_fs(url1)
print(f"Filesystem root is {fs1root}")
fs1.ls(fs1root)
print(url)



Filesystem root is janelia-flylight-imagery/Fly-eFISH/EASI-FISH_NP_SS
https://janelia-flylight-imagery.s3.amazonaws.com/Fly-eFISH/EASI-FISH_NP_SS
<class 's3fs.core.S3FileSystem'>


In [10]:
print(fs1.url('janelia-flylight-imagery'))
dir(fs1)


ParamValidationError: Parameter validation failed:
Invalid length for parameter Key, value: 0, valid min length: 1

In [4]:
url2 = "s3://janelia-flylight-imagery-dev/Fly-eFISH"
fs2, fs2root = get_fs(url2)
print(fs2root)
fs2.ls(fs2root, detail=True)

janelia-flylight-imagery-dev/Fly-eFISH


[{'Key': 'janelia-flylight-imagery-dev/Fly-eFISH/',
  'LastModified': datetime.datetime(2024, 2, 23, 15, 40, 52, tzinfo=tzutc()),
  'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
  'Size': 0,
  'StorageClass': 'STANDARD',
  'type': 'file',
  'size': 0,
  'name': 'janelia-flylight-imagery-dev/Fly-eFISH/'},
 {'Key': 'janelia-flylight-imagery-dev/Fly-eFISH/NP01_1_1_SS00790.json',
  'LastModified': datetime.datetime(2024, 2, 29, 20, 59, 29, tzinfo=tzutc()),
  'ETag': '"eb8fd5db7b1bfdba7c3a076f23ed9643"',
  'Size': 3459,
  'StorageClass': 'STANDARD',
  'type': 'file',
  'size': 3459,
  'name': 'janelia-flylight-imagery-dev/Fly-eFISH/NP01_1_1_SS00790.json'},
 {'Key': 'janelia-flylight-imagery-dev/Fly-eFISH/state.json',
  'LastModified': datetime.datetime(2024, 2, 28, 20, 32, 8, tzinfo=tzutc()),
  'ETag': '"7db26713fa2b049c3508145aed6dfec4"',
  'Size': 1101,
  'StorageClass': 'STANDARD',
  'type': 'file',
  'size': 1101,
  'name': 'janelia-flylight-imagery-dev/Fly-eFISH/state.json'},
 {'Key': 

In [1]:
url2 = "https://janelia-flylight-imagery-dev.s3.amazonaws.com/Fly-eFISH"
fs2, fs2root = get_fs(url2)
print(fs2root)
len(fs2.ls(fs2root, detail=True))

NameError: name 'get_fs' is not defined

In [37]:
url2 = "https://janelia-flylight-imagery-dev.s3.amazonaws.com/Fly-eFISH/state.json"
fs2, fs2root = get_fs(url2)
fs2.info(fs2root)

{'Key': 'janelia-flylight-imagery-dev/Fly-eFISH/state.json',
 'LastModified': datetime.datetime(2024, 2, 28, 20, 32, 8, tzinfo=tzutc()),
 'ETag': '"7db26713fa2b049c3508145aed6dfec4"',
 'Size': 1101,
 'StorageClass': 'STANDARD',
 'type': 'file',
 'size': 1101,
 'name': 'janelia-flylight-imagery-dev/Fly-eFISH/state.json'}

In [38]:
#https://filesystem-spec.readthedocs.io/en/latest/api.html#implementations
fsspec.available_protocols()

['data',
 'file',
 'local',
 'memory',
 'dropbox',
 'http',
 'https',
 'zip',
 'tar',
 'gcs',
 'gs',
 'gdrive',
 'sftp',
 'ssh',
 'ftp',
 'hdfs',
 'arrow_hdfs',
 'webhdfs',
 's3',
 's3a',
 'wandb',
 'oci',
 'ocilake',
 'asynclocal',
 'adl',
 'abfs',
 'az',
 'cached',
 'blockcache',
 'filecache',
 'simplecache',
 'dask',
 'dbfs',
 'github',
 'git',
 'smb',
 'jupyter',
 'jlab',
 'libarchive',
 'reference',
 'generic',
 'oss',
 'webdav',
 'dvc',
 'hf',
 'root',
 'dir',
 'box',
 'lakefs']

In [39]:
print(fsspec.get_filesystem_class("s3").__doc__)


    Access S3 as if it were a file system.

    This exposes a filesystem-like API (ls, cp, open, etc.) on top of S3
    storage.

    Provide credentials either explicitly (``key=``, ``secret=``) or depend
    on boto's credential methods. See botocore documentation for more
    information. If no credentials are available, use ``anon=True``.

    Parameters
    ----------
    anon : bool (False)
        Whether to use anonymous connection (public buckets only). If False,
        uses the key/secret given, or boto's credential resolver (client_kwargs,
        environment, variables, config files, EC2 IAM server, in that order)
    endpoint_url : string (None)
        Use this endpoint_url, if specified. Needed for connecting to non-AWS
        S3 buckets. Takes precedence over `endpoint_url` in client_kwargs.
    key : string (None)
        If not anonymous, use this access key ID, if specified. Takes precedence
        over `aws_access_key_id` in client_kwargs.
    secret : string (

In [40]:
import os
import json

def find_ngff(fs, root, path, depth=0):
    if depth>10: return []
    indent = depth * '  '
    #name = os.path.basename(path)
    name = os.path.relpath(path, start=root)
    #print(indent+path+" "+name)
    children = fs.ls(path, detail=True)
    child_names = [os.path.basename(c['name']) for c in children]
    if '.zattrs' in child_names:
        with fsspec.open(path+'/.zattrs') as f:
            attrs = json.load(f)
            if 'multiscales' in attrs:
                print(indent+'ZARR MULTISCALE '+name)
            if 'bioformats2raw.layout' in attrs:
                print(indent+'bioformats2raw series '+name)

    if '.zarray' in child_names:
        print(indent+'ZARR ARRAY '+name)
        return
    
    if 'attributes.json' in child_names:
        with fsspec.open(path+'/attributes.json') as f:
            attrs = json.load(f)
            if 'scales' in attrs:
                print(indent+'N5 MULTISCALE '+name)
            elif 'dimensions' in attrs:
                print(indent+'N5 ARRAY '+name)
                return
            elif 'n5' in attrs:
                #re.match("^c\d+$", "cf")

                print(indent+'N5 '+name)

    for d in [i['name'] for i in children if i['type']=='directory']:
        find_ngff(fs, root, d, depth+1)
        
find_ngff(fs1, fs1root, fs1root)

  N5 N2_352-1.n5
        N5 ARRAY N2_352-1.n5/image/c0/s0
  bioformats2raw series NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.czi.zarr
    ZARR MULTISCALE NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.czi.zarr/0
      ZARR ARRAY NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.czi.zarr/0/0
      ZARR ARRAY NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.czi.zarr/0/1
      ZARR ARRAY NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.czi.zarr/0/3
      ZARR ARRAY NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.czi.zarr/0/2
  N5 n5_ANM525849
    N5 MULTISCALE n5_ANM525849/c0
      N5 ARRAY n5_ANM525849/c0/s2
      N5 ARRAY n5_ANM525849/c0/s1
      N5 ARRAY n5_ANM525849/c0/s0
    N5 MULTISCALE n5_ANM525849/c1
      N5 ARRAY n5_ANM525849/c1/s2
      N5 ARRAY n5_ANM525849/c1/s1
      N5 ARRAY n5_ANM525849/c1/s0
    ZARR MULTISCALE N2_352-1.zarr/image
      ZARR ARRAY N2_352-1.zarr/image/s0


In [41]:
from typing import List, Union, Optional, Any, Dict, Literal
from enum import Enum
from pydantic import BaseModel, Field, ConfigDict
from typing_extensions import Annotated




In [42]:
import os
import json

def find_ome_zarrs(fs, root, path, depth=0):
    if depth>10: return []
    indent = depth * '  '
    name = os.path.relpath(path, start=root)
    children = fs.ls(path, detail=True)
    child_names = [os.path.basename(c['name']) for c in children]
    if '.zattrs' in child_names:
        with fsspec.open(path+'/.zattrs') as f:
            attrs = json.load(f)
            if 'multiscales' in attrs:
                print(indent+'ZARR MULTISCALE '+name)
            if 'bioformats2raw.layout' in attrs:
                print(indent+'bioformats2raw series '+name)

    if '.zarray' in child_names:
        print(indent+'ZARR ARRAY '+name)
        return
    

    for d in [i['name'] for i in children if i['type']=='directory']:
        find_ome_zarrs(fs, root, d, depth+1)
        
find_ome_zarrs(fs1, fs1root, fs1root)

  bioformats2raw series NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.czi.zarr
    ZARR MULTISCALE NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.czi.zarr/0
      ZARR ARRAY NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.czi.zarr/0/0
      ZARR ARRAY NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.czi.zarr/0/1
      ZARR ARRAY NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.czi.zarr/0/3
      ZARR ARRAY NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.czi.zarr/0/2
    ZARR MULTISCALE N2_352-1.zarr/image
      ZARR ARRAY N2_352-1.zarr/image/s0


In [43]:
"https://docs.aws.amazon.com/AmazonS3/latest/API/API_ListObjects.html#API_ListObjects_ResponseSyntax"

"Content-Type: binary/octet-stream"

"""
<?xml version="1.0" encoding="UTF-8"?>
<Error><Code>NoSuchKey</Code><Message>The specified key does not exist.</Message><Key>Fly-eFISH/NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.chunked.zarr/0/labels/.zattrs</Key><RequestId>JMPQ1066EVC7NQVY</RequestId><HostId>MYYlrY4+RSd8W5oLgypv4YP7T5G4V6nBoDL9sHFmIxdZVumSGudtAKW4COr18AFnbb1Fc5bBjVg36oDxHXqLCw==</HostId></Error>
"""

'\n<?xml version="1.0" encoding="UTF-8"?>\n<Error><Code>NoSuchKey</Code><Message>The specified key does not exist.</Message><Key>Fly-eFISH/NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.chunked.zarr/0/labels/.zattrs</Key><RequestId>JMPQ1066EVC7NQVY</RequestId><HostId>MYYlrY4+RSd8W5oLgypv4YP7T5G4V6nBoDL9sHFmIxdZVumSGudtAKW4COr18AFnbb1Fc5bBjVg36oDxHXqLCw==</HostId></Error>\n'

In [44]:

class Image(BaseModel):
    model_config = ConfigDict(extra='forbid') 
    id: str = Field(title="Id", description="Id for the data set container (unique within the parent folder)")
    path: str = Field(title="Path", description="Path to the container, relative to the overall root")
    axes: str = Field(title="Axes", description="Axes ")
    num_channels: int = Field(title="Num Channels", description="Number of channels in the image")
    num_timepoints: int = Field(title="Num Timepoints", description="Number of timepoints in the image")
    dimensions: str = Field(title="Dimensions", description="Size of the whole data set in nanometers")
    dimensions_voxels: str = Field(title="Dimensions (voxels)", description="Size of the whole data set in voxels")
    chunk_size: str = Field(title="Chunk size", description="Size of Zarr chunks")
    voxel_sizes: str = Field(title="Voxel Size", description="Size of voxels in nanometers. XYZ ordering.")
    compression: str = Field(title="Compression", description="Description of the compression used on the image data")


def encode_image(id, url, image_group):
    multiscales = image_group.attrs['multiscales']
    # TODO: what to do if there are multiple multiscales?
    multiscale = multiscales[0]
    axes = multiscale['axes']

    # Use highest resolution 
    dataset = multiscale['datasets'][0]
    array = image_group[image_group.name+'/'+dataset['path']]
    
    # TODO: shouldn't assume a single transform
    scale = dataset['coordinateTransformations'][0]['scale']

    axes_names = []
    dimensions_voxels = []
    voxel_sizes = []
    dimensions = []
    chunks = []
    num_channels = 1
    num_timepoints = 1
    for i, axis in enumerate(axes):
        axes_names.append(axis['name'].upper())
        unit = ''
        if axis['type']=='space':
            unit = axis['unit']
            if unit=='micrometer': unit = " μm"
            if unit=='nanometer': unit = " nm"
            voxel_sizes.append("%.2f%s" % (round(scale[i],2), unit))
            dimensions.append("%.2f%s" % (round(array.shape[i] * scale[i],2), unit))
        elif axis['type']=='channel':
            num_channels = array.shape[i]
            voxel_sizes.append("%i" % scale[i])
            dimensions.append("%i" % (array.shape[i] * scale[i]))
        elif axis['type']=='time':
            num_timepoints = array.shape[i]
            voxel_sizes.append("%i" % scale[i])
            dimensions.append("%i" % (array.shape[i] * scale[i]))
        dimensions_voxels.append(str(array.shape[i]))
        chunks.append("%i" % array.chunks[i])

    return Image(
        id = id,
        path = url,
        axes = ''.join(axes_names),
        num_channels = num_channels,
        num_timepoints = num_timepoints,
        voxel_sizes = ' ✕ '.join(voxel_sizes),
        dimensions = ' ✕ '.join(dimensions),
        dimensions_voxels = ' ✕ '.join(dimensions_voxels),
        chunk_size = ' ✕ '.join(chunks),
        compression = str(array.compressor)
    )

import zarr
url = "./data/NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.czi.zarr/0"
z = zarr.open(url, mode='r')

image = encode_image('01', url, z)
image

Image(id='01', path='./data/NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.czi.zarr/0', axes='TCZYX', num_channels=4, num_timepoints=1, dimensions='1 ✕ 4 ✕ 376.00 μm ✕ 447.29 μm ✕ 447.29 μm', dimensions_voxels='1 ✕ 4 ✕ 752 ✕ 1920 ✕ 1920', chunk_size='1 ✕ 1 ✕ 1 ✕ 1920 ✕ 1920', voxel_sizes='1 ✕ 1 ✕ 0.50 μm ✕ 0.23 μm ✕ 0.23 μm', compression="Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)")

In [45]:
import itertools
import zarr
    
def yield_images_nested(z):
    # This only works with storage backends that support listing items like 
    # local disk and S3, but not HTTP for example.
    for _,group in z.groups():
        if 'multiscales' in group.attrs:
            yield group
        for image in yield_images_nested(group):
            yield image

def yield_images(url):
    ''' Interrogates the OME-Zarr at the given URL and yields all of the 2-5D images within.
    '''
    z = zarr.open(url, mode='r')
    # Based on https://ngff.openmicroscopy.org/latest/#bf2raw
    if 'bioformats2raw.layout' in z.attrs and z.attrs['bioformats2raw.layout']==3:
        if 'OME' in z:
            series = z['OME'].attrs['series']
            if len(series) == 1:
                # We treat this as a single image for easier consumption
                yield z[series[0]]
            else:
                # Spec: "series" MUST be a list of string objects, each of which is a path to an image group.
                for image_id in series:
                    yield z[image_id]
        else:
            # Spec: If the "series" attribute does not exist and no "plate" is present:
            # - separate "multiscales" images MUST be stored in consecutively numbered groups starting from 0 (i.e. "0/", "1/", "2/", "3/", ...).
            for i in itertools.count():
                try:
                    yield z[str(i)]
                except:
                    break
    elif 'multiscales' in z.attrs:
        yield z
    else:
        for image in yield_images_nested(z):
            yield image

def get_images(url):
    print(f"Getting images from {url}")
    for image_group in yield_images(url):
        image = encode_image(url, url, image_group)
        print(image.__repr__())


url = "./data/NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.czi.zarr"
get_images(url)
#%timeit get_images(url)
#812 µs ± 11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Getting images from ./data/NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.czi.zarr
Image(id='./data/NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.czi.zarr', path='./data/NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.czi.zarr', axes='TCZYX', num_channels=4, num_timepoints=1, dimensions='1 ✕ 4 ✕ 376.00 μm ✕ 447.29 μm ✕ 447.29 μm', dimensions_voxels='1 ✕ 4 ✕ 752 ✕ 1920 ✕ 1920', chunk_size='1 ✕ 1 ✕ 1 ✕ 1920 ✕ 1920', voxel_sizes='1 ✕ 1 ✕ 0.50 μm ✕ 0.23 μm ✕ 0.23 μm', compression="Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)")


In [46]:

url = "https://janelia-flylight-imagery-dev.s3.amazonaws.com/Fly-eFISH/NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.chunked.zarr"
get_images(url)
#%timeit get_images(url)
#365 ms ± 51.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Getting images from https://janelia-flylight-imagery-dev.s3.amazonaws.com/Fly-eFISH/NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.chunked.zarr
Image(id='https://janelia-flylight-imagery-dev.s3.amazonaws.com/Fly-eFISH/NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.chunked.zarr', path='https://janelia-flylight-imagery-dev.s3.amazonaws.com/Fly-eFISH/NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.chunked.zarr', axes='TCZYX', num_channels=4, num_timepoints=1, dimensions='1 ✕ 4 ✕ 376.00 μm ✕ 447.29 μm ✕ 447.29 μm', dimensions_voxels='1 ✕ 4 ✕ 752 ✕ 1920 ✕ 1920', chunk_size='1 ✕ 1 ✕ 128 ✕ 128 ✕ 128', voxel_sizes='1 ✕ 1 ✕ 0.50 μm ✕ 0.23 μm ✕ 0.23 μm', compression="Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)")


In [47]:
z = zarr.open(url, mode='r')

In [53]:
z['0'].attrs['omero']

{'channels': [{'color': '00FF00',
   'coefficient': 1,
   'active': True,
   'label': 'Cam1-T1',
   'window': {'min': 40.0, 'max': 51986.0, 'start': 40.0, 'end': 51986.0},
   'family': 'linear',
   'inverted': False},
  {'color': 'FF00FF',
   'coefficient': 1,
   'active': True,
   'label': 'Cam2-T1',
   'window': {'min': 52.0, 'max': 16528.0, 'start': 52.0, 'end': 16528.0},
   'family': 'linear',
   'inverted': False},
  {'color': 'FF0000',
   'coefficient': 1,
   'active': True,
   'label': 'Cam2-T2',
   'window': {'min': 62.0, 'max': 32500.0, 'start': 62.0, 'end': 32500.0},
   'family': 'linear',
   'inverted': False},
  {'color': '00FFFF',
   'coefficient': 1,
   'active': False,
   'label': 'Cam1-T3',
   'window': {'min': 78.0, 'max': 9636.0, 'start': 78.0, 'end': 9636.0},
   'family': 'linear',
   'inverted': False}],
 'rdefs': {'defaultT': 0, 'model': 'color', 'defaultZ': 376}}

In [1]:
import json
from pydantic_neuroglancer.viewer_state import ViewerState

with open('state.json', 'r') as f:
    state_json = json.load(f)
    state = ViewerState(**state_json)

state

/Users/rokickik/opt/anaconda3/envs/zarrcade/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.generics:GenericModel` has been moved to `pydantic.BaseModel`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


ValidationError: 19 validation errors for ViewerState
position
  Tuple should have at most 3 items after validation, not 4 [type=too_long, input_value=[396, 912, 976, 0], input_type=list]
    For further information visit https://errors.pydantic.dev/2.6/v/too_long
layers.0.image.shaderControls.normalized.float
  Input should be a valid number [type=float_type, input_value={'range': [185, 5032]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/float_type
layers.0.image.shaderControls.normalized.InvlerpParameters.window
  Field required [type=missing, input_value={'range': [185, 5032]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
layers.0.image.localDimensions
  Extra inputs are not permitted [type=extra_forbidden, input_value={"c'": [1, '']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/extra_forbidden
layers.0.image.localPosition
  Extra inputs are not permitted

In [32]:
from neuroglancer.viewer_state import ViewerState, CoordinateSpace, ImageLayer

state = ViewerState()
state.dimensions = CoordinateSpace(
    names=['z','y','x','t'], 
    scales=[5e-7, 2.329612336924942e-7, 2.329612336924942e-7, 1], 
    units=['m','m','m',''])
state.position = [396, 912, 976, 0]
state.crossSectionScale = 4.687971627022003
state.projectionScale = 2048

state.layers.append(
    name='ch0',
    layer=ImageLayer(
        source='zarr://s3://janelia-flylight-imagery-dev/Fly-eFISH/NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.chunked.zarr/0',
        layerDimensions=CoordinateSpace(names=["c'"], scales=[1], units=['']),
        localPosition=[0],
        tab='rendering',
        opacity=1,
        blend='additive',
        shader='#uicontrol vec3 hue color(default=\"red\")\n#uicontrol invlerp normalized(range=[0,4096])\nvoid main(){emitRGBA(vec4(hue*normalized(),1));}',
        shaderControls={
            'normalized': {
                'range': [185, 5032]
            }
        }
    )
)

state.layout = '4panel'
state.to_json()


{'dimensions': {'z': [5e-07, 'm'],
  'y': [2.329612336924942e-07, 'm'],
  'x': [2.329612336924942e-07, 'm'],
  't': [1.0, '']},
 'position': [396.0, 912.0, 976.0, 0.0],
 'crossSectionScale': 4.687971627022003,
 'projectionScale': 2048.0,
 'layers': [{'type': 'image',
   'source': [{'url': 'zarr://s3://janelia-flylight-imagery-dev/Fly-eFISH/NP01_1_1_SS00790_AstA546_CCHa1_647_1x_LOL.chunked.zarr/0'}],
   'localDimensions': {"c'": [1.0, '']},
   'localPosition': [0.0],
   'tab': 'rendering',
   'opacity': 1.0,
   'blend': 'additive',
   'shader': '#uicontrol vec3 hue color(default="red")\n#uicontrol invlerp normalized(range=[0,4096])\nvoid main(){emitRGBA(vec4(hue*normalized(),1));}',
   'shaderControls': {'normalized': {'range': [185, 5032]}},
   'name': 'ch0'}],
 'layout': '4panel'}

In [8]:
state.dimensions

CoordinateSpace({'z': [5e-07, 'm'], 'y': [2.329612336924942e-07, 'm'], 'x': [2.329612336924942e-07, 'm'], 't': [1.0, '']})

In [11]:
state.to_json()

{'dimensions': {'z': [5e-07, 'm'],
  'y': [2.329612336924942e-07, 'm'],
  'x': [2.329612336924942e-07, 'm'],
  't': [1.0, '']}}

In [7]:
import zarr
path = '/nrs/scicompsoft/rokicki/multifish/demo_tiny/outputs/LHA3_R3_tiny/stitching/export.n5'
z = zarr.open(store=zarr.N5FSStore(path), mode='r')

z.attrs




In [9]:
[x for x in z.attrs]
z.attrs['scales']

[[1.0, 1.0, 1.0],
 [2.0, 2.0, 1.0],
 [4.0, 4.0, 2.0],
 [8.0, 8.0, 4.0],
 [16.0, 16.0, 9.0],
 [32.0, 32.0, 18.0],
 [64.0, 64.0, 35.0],
 [128.0, 128.0, 70.0],
 [256.0, 256.0, 140.0]]